In [2]:
import numpy as np
import yaml
from simulator import simulate

with open('examples.yaml') as f:
    examples = yaml.load(f, Loader=yaml.CLoader)

In [3]:
def print_align_As(x1 : str, x2 : str):
    x1l = x1.split('A')
    x2l = x2.split('A')
    x1out = []
    x2out = []
    for i in range(max(len(x1l), len(x2l))):
        if i >= len(x1l):
            x1s = ' '*len(x2l[i])
        else:
            x1s = x1l[i]
        if i >= len(x2l):
            x2s = ' '*len(x1l[i])
        else:
            x2s = x2l[i]

        if len(x1s) < len(x2s):
            x1s += ' '*(len(x2s) - len(x1s))
        if len(x2s) < len(x1s):
            x2s += ' '*(len(x1s) - len(x2s))

        x1out.append(x1s)
        x2out.append(x2s)

    print('A'.join(x1out))
    print('A'.join(x2out))

In [4]:
PRESS = 'A'
LEFT = '<'
RIGHT = '>'
UP = '^'
DOWN = 'v'

NUMERIC_KEYPAD = {
    PRESS : (2, 0),
    '0' : (1, 0),
    '1' : (0, 1),
    '2' : (1, 1),
    '3' : (2, 1),
    '4' : (0, 2),
    '5' : (1, 2),
    '6' : (2, 2),
    '7' : (0, 3),
    '8' : (1, 3),
    '9' : (2, 3)
}

DIRECTIONAL_KEYPAD = {
    LEFT : (0,0),
    DOWN : (1,0),
    RIGHT : (2,0),
    UP : (1,1),
    PRESS : (2, 1)
}

DIRECTIONAL_KEYPAD_ACTIONS = {
    LEFT : (-1, 0),
    RIGHT : (1, 0),
    UP : (0, 1),
    DOWN : (0, -1)
}

In [30]:
def unsorted_cost(lst : list):    
    bigger = 0
    smaller = 0
    for l0, l1 in zip(lst[:-1], lst[1:]):
        if l1 > l0:
            bigger += 1
        elif l1 < l0:
            smaller += 1

    return abs(bigger - smaller)

    

    




def sequence(input_sequence, numeric : bool = False):
    keypad = NUMERIC_KEYPAD if numeric else DIRECTIONAL_KEYPAD
    p = np.array(keypad[PRESS])
    actions = ''
    for i, s in enumerate(input_sequence):
        next_p = np.array(keypad[s])
        v = next_p - p
        x, y = v

        left_right = (LEFT*(-x) if x < 0 else RIGHT*x)
        up_down = (DOWN*(-y) if y < 0 else UP*y)

        
        left_right_first = left_right + up_down
        left_right_first_cost = [DIRECTIONAL_KEYPAD_ACTIONS[x][0] for x in left_right_first]

        up_down_first = up_down + left_right
        up_down_first_cost = [DIRECTIONAL_KEYPAD_ACTIONS[x][0] for x in up_down_first]

        #= not (len(left_right) > len(up_down))

        if unsorted_cost(left_right_first_cost) > unsorted_cost(up_down_first_cost):
            actions += left_right + up_down
        else:
            actions += up_down + left_right

        actions += PRESS

        p = next_p

    return actions


In [31]:
def sequence_compare(x1, x2):
    if x1 == x2:
        return '✅'
    elif len(x1) == len(x2):
        return '⚠'
    else:
        return '❌'

PRINT_ON = '❌' #+ '⚠' + '✅'

for s in examples:
    # if s != '456A':
    #     continue
    print(s)
    A1 = examples[s][0]
    A2 = sequence(s, True)
    result = sequence_compare(A1, A2)
    print(f'A : {result}')
    if result in PRINT_ON:
        print_align_As(A1, A2)
    B1 = examples[s][1]
    B2 = sequence(A2)
    result = sequence_compare(B1, B2)
    print(f'B : {result}')
    if result in PRINT_ON:
        print_align_As(B1, B2)
    C1 = examples[s][2]
    C2 = sequence(B2)
    result = sequence_compare(C1, C2)
    print(f'C : {result}')
    if result in PRINT_ON:
        print_align_As(C1, C2)
        # print(C1)
        # print(C2)

    #print(simulate(C2))

029A
A : ⚠
B : ⚠
C : ⚠
980A
A : ✅
B : ⚠
C : ⚠
179A
A : ✅
B : ⚠
C : ⚠
456A
A : ✅
B : ⚠
C : ⚠
379A
A : ⚠
B : ⚠
C : ❌
<v<A>>^AvA^A<v A<  AA>>^A Av  A<^A>A  AvA^A<vA>^AA<A>A<v<A>A>^ AAAvA<^A>A
v<<A^>>AvA^Av<<A^>>AAv< A<A^>>A  A<Av>A A^Av<A^>AA<A>Av< A<A^>>AAA<Av>A^A
